In [1]:
import pandas as pd
import numpy as np
import os
from glob import glob
from collections import Counter
os.chdir('/Users/xiaodiu/Documents/github/JudgeSentences/')

In [ ]:
df = pd.DataFrame()
itr = pd.read_stata('rawdata/sentences/opafy00nid.dta',iterator=True)
chunk = itr.get_chunk(10000)
df = df.append(chunk, ignore_index=True)

In [ ]:

from glob import glob

import pandas as pd
import re


folders = glob('*')

num = 0
for folder in folders:        
    members = glob(folder + '/*txt')
    num += len(members)
    
print "total %d" %(num)

In [ ]:
case2judge = pd.read_pickle('rawdata/cluster2songername.pkl')
casefreq_akd = pd.read_pickle('rawdata/case_level/casefreqs-akd.pkl')
id2word = pd.read_pickle('rawdata/id2word-2.pkl')
word2id = pd.read_pickle('rawdata/word2id-2.pkl')
df_Holger = pd.read_stata('rawdata/Holger_matched_USSC_TRAC2.dta')
df_Holger_demean = pd.read_stata('rawdata/Holger_matched_USSC_TRAC2_include_demean.dta')
df_Trac_USSC_Judges = pd.read_stata('rawdata/Trac_USSC_JudgesBIOs.dta')
docfreq_akd = pd.read_pickle('rawdata/text/docfreqs-akd.pkl')

In [ ]:
case2judge = pd.read_pickle('rawdata/cluster2songername.pkl')
judge2case = dict(zip(case2judge.values(),case2judge.keys()))
files = glob('rawdata/case_level/*.pkl')
num = len(files)
judge2count_bigram = Counter()
i = 1
for file in files:
    #print('file :'+ file)
    case_freq = pd.read_pickle(file)
    for k,v in case_freq.items():
        k = int(k)
        if k in case2judge:
            judge = case2judge[k]
            #print('judge_name:' + judge)
            if(judge2count_bigram[judge] == 0):
                judge2count_bigram[judge] = Counter()
            judge2count_bigram[judge] += v
    if(i % 20 == 0 ):
        print('finished : %s' % (i))
    i += 1

In [ ]:
judge2exist_bigram = Counter()
for k, v in judge2count_bigram.items():
    judge2exist_bigram[k] = Counter(v.keys())

$$
y_i = \alpha Judge_i + \beta X + \alpha_2type + \epsilon_i
$$

In [ ]:
pd.to_pickle(judge2exist_bigram,
             'rawdata/judge2exist_bigram.pkl')
pd.to_pickle(judge2count_bigram,
             'rawdata/judge2count_bigram.pkl')

In [ ]:
df_Trac_USSC_Judges = pd.read_stata('rawdata/Trac_USSC_JudgesBIOs.dta')

In [ ]:
Holger_matched_USSC = pd.read_stata('rawdata/Holger_matched_USSC_TRAC2.dta')

In [ ]:
Holger_matched_USSC.loc[:,['statmin','statmax','xmaxsor','xminsor','glmin','glmax','crime','prison','senttot']]


In [ ]:
## Creat training and testing datasets.

In [ ]:
Holger_matched_USSC.columns

In [ ]:
Holger_matched_USSC = pd.read_stata('rawdata/Holger_matched_USSC_TRAC2.dta')

In [ ]:
columns = ['district','sentyr','trial','monrace', 'monsex', 'newcit','newrace','nocounts',
           'statmin','statmax','xmaxsor','xminsor','glmin','glmax','crime','prison','senttot',
           'senttot0','judgeid','judge_yearsonbench','malejudge']
Holger_matched = Holger_matched_USSC.loc[:, columns]

In [ ]:
Holger_matched

In [ ]:
Holger_matched = Holger_matched[pd.notnull(Holger_matched['statmax'])]
Holger_matched = Holger_matched[pd.notnull(Holger_matched['statmin'])]
Holger_matched = Holger_matched[pd.notnull(Holger_matched['prison'])]
Holger_matched['Harshness'] = (Holger_matched.prison - Holger_matched.statmin)
Holger_matched.shape

In [ ]:
group_Holger = Holger_matched.loc[:,['district','sentyr','Harshness']].groupby(['district','sentyr'])

In [ ]:
demean = lambda df: df - df.mean()
group_Holger.transform(demean)

In [ ]:
res = Holger_matched.loc[:,['district','sentyr']]

In [ ]:
Holger_matched.sentyr.describe()

In [ ]:
### opinion id to judge year


In [ ]:
import re
import os
from collections import Counter
import datetime

In [ ]:
case2judge = pd.read_pickle('datasets/cluster2songername.pkl')

In [ ]:
judges = np.unique(list(case2judge.values()))
judge2case = [[v,k] for k, v in case2judge.items()]

In [ ]:
judge2year_case = {}
for judge in judges:
    cases = [v for k,v in judge2case if k == judge]
    year2case = {}
    for case in cases:
        try:
            year = case2year[str(case)]
            if year2case.get(year) == None:
                year2case[year] = []
            else:
                year2case[year].append(case)
        except KeyError as err:
            continue
    judge2year_case[judge] = year2case 
pd.to_pickle(judge2year_case,'datasets/judge2year_case.pkl')

In [ ]:
text = open('rawdata/text/almd/1651141.txt').read()

In [ ]:
normtext = re.sub('[^a-z0-9 ]',' ',text.lower())

In [ ]:
p = re.compile(r'\d{1,2}?  \d{4}?')
case2year = {}
folders = glob('rawdata/text/*')
for folder in folders:
    members = glob(folder + '/*txt')
    print(folder,len(members))
    if len(members) == 0:
        continue
    for fname in members:
        if not fname.endswith('txt'):
            continue
        caseid = fname.split('/')[-1][:-4]
        text = open(fname).read()    
        normtext = re.sub('[^a-z0-9 ]',' ',text.lower())
        years = p.findall(normtext)
        proyear = []
        for year in years:
            try :
                date = datetime.datetime.strptime(year,"%d  %Y")
                if date.year > 1900 and date.year < 2017:
                    proyear.append(date.year)
            except ValueError as err:
                continue
        if len(proyear) > 0:
            case2year[caseid] = np.max(proyear)
pd.to_pickle(case2year,'datasets/case2year.pkl')

In [ ]:
### tokens set , bigrams set ,trigrams set, four grams
# only take the opinion of range 2002 -2017

In [2]:
import os
from glob import glob
from nltk import bigrams as bigramize, ngrams,word_tokenize, PorterStemmer, trigrams
import pandas as pd
import re
from collections import Counter
from zipfile import ZipFile
from random import shuffle
porter = PorterStemmer()
from nltk.corpus import stopwords
stoplist = set(porter.stem(w) for w in stopwords.words('english'))


In [3]:
case2year = pd.read_pickle('datasets/case2year.pkl')
judge2year_case = pd.read_pickle('datasets/judge2year_case.pkl')
opinion2floder = pd.read_pickle('datasets/opinion2floder.pkl')

In [4]:
bigrams_dict = Counter()
tokens_dict = Counter()
trigrams_dict = Counter()
fourgrams_dict = Counter()
fivegrams_dict = Counter()

judges = judge2year_case.keys()
for judge in judges:
    lens = 0
    opinions = judge2year_case[judge]
    for year in opinions.keys():
        if year < 2002 or year > 2011:
            continue
        opilist = opinions[year]
        for opi in opilist:    
            fold = opinion2floder[str(opi)]
            filepath = 'rawdata/text/' + fold + '/' + str(opi) + '.txt'
            #print(filepath)
            text = open(filepath).read()    
            normtext = re.sub('[^a-z0-9 ]',' ',text.lower())
            tokens = normtext.split() 
            tokens = [porter.stem(w) for w in tokens if len(w) >=3 and w not in stoplist]      
            #bigramset = set(['_'.join(b) for b in bigramize(tokens)])
            #trigramset = set(['_'.join(b) for b in trigrams(tokens)])
            #fourgramset = set(['_'.join(b) for b in ngrams(tokens, 4)])
            fivegramset = set(['_'.join(b) for b in ngrams(tokens, 5)])
            
            tokens_dict.update(tokens)
            #bigrams_dict.update(bigramset)
            #trigrams_dict.update(trigramset)
            #fourgrams_dict.update(fourgramset)
            fivegrams_dict.update(fivegramset)
            #lens += len(fivegrams_dict)
    print(judge+str(len(fivegrams_dict)))

LOWE, MARY JOHNSON0
BRYANT, WILLIAM B.0
HUPP, HARRY L.0
BULLOCK, FRANK W., JR.194530
LEE, DONALD J., SR.194530
FREEDMAN, FRANK H.206773
FISCHER, DALE SUSAN270517
MARTIN, JOHN S., JR.271447
SCHILTZ, PATRICK JOSEPH579486
LEE, TOM STEWART985166
MARTEN, J. THOMAS985166
LEGG, BENSON EVERETT1147582
COVELLO, ALFRED VINCENT1214014
METZNER, CHARLES M.1214014
CLARKE, JOSEPH CALVITT, JR.1214014
MAGNUS-STINSON, JANE ELIZABETH1239167
BRODY, ANITA B.1987472
WRIGHT, SCOTT O.1992186
FROST, GREGORY L.2244566
BUNTON, LUCIUS DESHA, III2244566
GELPI, GUSTAVO ANTONIO, JR.2380110
GREENAWAY, JOSEPH A.2467364
GOLDBERG, MITCHELL S.2490204
DITTER, J. WILLIAM, JR.2490204
VIKEN, JEFFREY LYNN2513617
CONTI, SAMUEL2652731
MARSH, MALCOLM FRANCIS2658909
JOHNSON, WILLIAM PAUL2735622
TAGLE, HILDA G.2754022
HAWKINS, FALCON B., JR.2754022
ROSENBLATT, PAUL G.2809985
FLANNERY, THOMAS A.2809985
HERLONG, HENRY M., JR.2842846
ROVNER, ILANA DIAMOND2842846
HENDERSON, THELTON EUGENE2908478
WOODCOCK, JOHN A., JR.3363832
LONGOBARDI

In [5]:
fivegrams_dict_common = fivegrams_dict.most_common(500000)

In [7]:
fivegrams = {k:v for k,v in fivegrams_dict_common}

In [8]:
fivegrams.popitem()

('thi_court_may_exercis_jurisdict', 63)

In [10]:
pd.to_pickle(fivegrams, 'datasets/grams_5.pkl')

In [ ]:
pd.to_pickle(tokens_dict, 'datasets/tokens.pkl')
pd.to_pickle(bigrams_dict, 'datasets/bigrams_dict.pkl')
pd.to_pickle(trigrams_dict, 'datasets/trigrams_dict.pkl')

In [ ]:
import os
from glob import glob
from nltk import bigrams as bigramize, word_tokenize, PorterStemmer, trigrams
import pandas as pd
import re
from collections import Counter
from zipfile import ZipFile
from random import shuffle

folders = glob('rawdata/text/*')

porter = PorterStemmer()
from nltk.corpus import stopwords
stoplist = set(porter.stem(w) for w in stopwords.words('english'))
lens = 0

threshold = 3
bigrams_dict = Counter()
tokens_dict = Counter()
trigrams_dict = Counter()

for folder in folders:        
    members = glob(folder + '/*txt')
    print(folder,len(members))
    if len(members) == 0:
        continue
    lens += len(members)
    for fname in members:
        if not fname.endswith('txt'):
            continue
        cluster = fname.split('/')[-1][:-4]    
        text = open(fname).read()    
        normtext = re.sub('[^a-z]',' ',text.lower())
        tokens = normtext.split()     
        tokens = [porter.stem(w) for w in tokens if len(w) >=3 and w not in stoplist]      
        bigramset = set(['_'.join(b) for b in bigramize(tokens)])
        trigramset = set(['_'.join(b) for b in trigrams(tokens)])
        
        tokens_dict.update(tokens)
        bigrams_dict.update(bigramset)
        trigrams_dict.update(trigramset)

tokens_dict = tokens_dict.most_common(50000)
bigrams_dict = bigrams_dict.most_common(200000)
trigrams_dict = trigrams_dict.most_common(200000)

In [ ]:
folders = glob('rawdata/text/*')
opinion2floder = {}

for folder in folders:        
    members = glob(folder + '/*txt')
    fold = folder.split('/')[-1]
    #print(fold,len(members))
    if len(members) == 0:
        continue
    lens += len(members)
    for fname in members:
        if not fname.endswith('txt'):
            continue
        cluster = fname.split('/')[-1][:-4] 
        opinion2floder[cluster] = fold

pd.to_pickle(opinion2floder, 'datasets/opinion2floder.pkl')

In [ ]:
## the judges who has no opinions 

In [ ]:
judges_with_no_opinions = []
judges = judge2year_case.keys()
for judge in judges:
    opinions = judge2year_case[judge]
    for year in opinions.keys():
        if year < 2002 or year > 2011:
            continue
        opilist = opinions[year]
        if opilist == []:
            judges_with_no_opinions.append([judge,year])
judges_with_no_opinions = pd.DataFrame(np.array(judges_with_no_opinions))
judges_with_no_opinions.to_csv('datasets/judgewithnoopinios.csv', index= False,header=False)

In [3]:
## give every grams a index
# grams include unigrams, bigrams, trigrams ,forgram and fivegrams
dic = 'datasets/grams_dict1978-2012/'
members = glob(dic + '*.pkl')

grams_len = 0
grams = []
for member in members:
    grams_n = pd.read_pickle(member)
    grams_len += len(grams_n)
    grams += list(grams_n.keys())
    print(grams_len)
index = np.random.permutation(range(grams_len))
grams_dict = dict(zip(grams, index))





100000
200000


In [4]:
pd.to_pickle(grams_dict, 'datasets/grams_dict1978-2012/grams_dict.pkl')

In [41]:
## give every judge_year a index
judge_year2index = Counter()
np.random.seed(0)
judges = judge2year_case.keys()
index = np.random.permutation(range(100000))
i = 0
for judge in judges:
    year2id = Counter()
    for y in range(1960,2018):
        year2id[y] = index[i]
        i += 1
    judge_year2index[judge] = year2id

   

In [45]:
pd.to_pickle(judge_year2index, 'datasets/judge_year2index.pkl')

In [ ]:
## take BOW features 
# format Index(judge_year) + feature map

In [25]:
import os
from glob import glob
from nltk import bigrams as bigramize, word_tokenize, PorterStemmer, trigrams
import pandas as pd
import re
from collections import Counter
from zipfile import ZipFile
from random import shuffle

folders = glob('rawdata/text/*')
#grams_dict = pd.read_pickle('datasets/grams_dict/grams_dict.pkl')

porter = PorterStemmer()
from nltk.corpus import stopwords
stoplist = set(porter.stem(w) for w in stopwords.words('english'))
lens = 0

bow_features = Counter()

judges = judge2year_case.keys()
for judge in judges:
    judge_grams = Counter()
    opinions = judge2year_case[judge]
    for year in opinions.keys():
        if year < 2002 or year > 2011:
            continue
        opilist = opinions[year]
        year_grams = Counter()
        for opi in opilist:    
            fold = opinion2floder[str(opi)]
            filepath = 'rawdata/text/' + fold + '/' + str(opi) + '.txt'
            #print(filepath)
            text = open(filepath).read()    
            normtext = re.sub('[^a-z0-9 ]',' ',text.lower())
            tokens = normtext.split() 
            tokens = [porter.stem(w) for w in tokens if len(w) >=3 and w not in stoplist]      
            tokens = [grams_dict[w] for w in tokens]
            year_grams.update(tokens)
        print(len(year_grams))
        judge_grams[year] = year_grams
    bow_features[judge] = judge_grams

0
3523
5162
6515
4879
534
0
0
0
0
2352
1966
1343
2647
0
3106
1197
2385
391
2041
6862
5103
6998
4353
2753
4637
4294
4311
4501
4289
5648
4476
5801
4294
3216
0
2477
0
2085
678
4899
2746
3753
3898
2895
2392
0
2198
4313
2654
0
509
2562
1700
7014
5617
6508
7153
6601
5401
5038
6100
6032
6728
0
0
380
0
0
0
523
762
0
794
3681
2919
4914
742
6117
5452
1371
1959
2488
2537
1188
2745
4997
3165
3124
1625
1015
2531
2897
3111
2915
2719
936
2379
1564
0
3285
999
1959
1078
1777
1062
778
3446
3586
4047
3749
0
0
1553
0
0
0
1195
1370
0
0
1965
1912
1536
2838
2963
1036
1019
1109
1589
1047
0
2046
1520
0
1348
3125
2332
0
0
858
0
0
823
1426
1756
2616
0
0
0
1516
0
0
0
1341
1244
1867
0
4569
1234
1869
7650
9127
7336
8644
0
0
0
2019
0
1185
826
0
7241
5839
4416
2800
3426
4245
5860
1628
1605
606
2538
1650
4277
2704
4139
2843
1654
2991
1490
2900
319
0
1056
0
0
2508
1960
1553
461
0
1120
0
1177
0
0
1469
1890
589
1163
621
1869
808
409
2930
2769
2601
4322
4410
2816
4051
3959
1833
2302
3796
5110
3156
1620
2836
2083
3139
1500